In [1]:
import argparse
import os
import sys
from pathlib import Path
import json
import matplotlib.pyplot as plt
%matplotlib inline

import imageio
import numpy as np
from tqdm import tqdm
try:
    import renderpy
except ImportError:
    print("renderpy not installed. Please install renderpy from https://github.com/liu115/renderpy")
    sys.exit(1)

In [2]:
from utils.colmap import read_model, write_model, Image
from scene_release import ScannetppScene_Release
from utils.utils import run_command, load_yaml_munch, load_json, read_txt_list

In [3]:
p = argparse.ArgumentParser()
p.add_argument("config_file", help="Path to config file", default="/home/kumaraditya/scannetpp/common/configs/render.yml", nargs="?")
args = p.parse_args([])

print(f"Config file: {args.config_file}")


Config file: /home/kumaraditya/scannetpp/common/configs/render.yml


In [4]:
cfg = load_yaml_munch(args.config_file)

# get the scenes to process
if cfg.get("scene_ids"):
    scene_ids = cfg.scene_ids
elif cfg.get("splits"):
    scene_ids = []
    for split in cfg.splits:
        split_path = Path(cfg.data_root) / "splits" / f"{split}.txt"
        scene_ids += read_txt_list(split_path)

output_dir = cfg.get("output_dir")
if output_dir is None:
    # default to data folder in data_root
    output_dir = Path(cfg.data_root) / "data"
output_dir = Path(output_dir)

render_devices = []
if cfg.get("render_dslr", False):
    render_devices.append("dslr")
    raise Exception("This code is has not been tested with the DSLR data.")
if cfg.get("render_iphone", False):
    render_devices.append("iphone")

In [5]:
scene_id = scene_ids[0]
print(scene_id)
print(render_devices)

c0f5742640
['iphone']


In [6]:
from render_crops_utils import vert_to_obj_lookup, CropHeap, crop_rgb_mask, plot_grid_images

scene = ScannetppScene_Release(scene_id, data_root=Path(cfg.data_root) / "data")
render_engine = renderpy.Render()
render_engine.setupMesh(str(scene.scan_mesh_path))

# Load annotations
segments_anno = json.load(open(scene.scan_anno_json_path, "r"))
n_objects = len(segments_anno["segGroups"])
instance_colors = np.random.randint(low=0, high=256, size=(n_objects + 1, 3), dtype=np.uint8)
instance_colors[0] = 255 # White bg
vert_to_obj = vert_to_obj_lookup(segments_anno)

# Crop heaps
crop_heaps = dict()
for obj in segments_anno["segGroups"]:
    crop_heaps[obj["id"]] = dict()
    crop_heaps[obj["id"]]["label"] = obj["label"]
    crop_heaps[obj["id"]]["heap"] = CropHeap(max_size=4)

# Background class is 0
assert 0 not in crop_heaps
crop_heaps[0] = dict()
crop_heaps[0]["label"] = "BACKGROUND"
crop_heaps[0]["heap"] = CropHeap(max_size=4)


for device in render_devices:
    if device == "dslr":
        cameras, images, points3D = read_model(scene.dslr_colmap_dir, ".txt")
    else:
        cameras, images, points3D = read_model(scene.iphone_colmap_dir, ".txt")
    assert len(cameras) == 1, "Multiple cameras not supported"
    camera = next(iter(cameras.values()))

    fx, fy, cx, cy = camera.params[:4]
    params = camera.params[4:]
    camera_model = camera.model
    render_engine.setupCamera(
        camera.height, camera.width,
        fx, fy, cx, cy,
        camera_model,
        params,      # Distortion parameters np.array([k1, k2, k3, k4]) or np.array([k1, k2, p1, p2])
    )

    near = cfg.get("near", 0.05)
    far = cfg.get("far", 20.0)
    rgb_dir = Path(cfg.output_dir) / scene_id / device / "render_rgb"
    depth_dir = Path(cfg.output_dir) / scene_id / device / "render_depth"
    crop_dir = Path(cfg.output_dir) / scene_id / device / "render_crops_kumar_w_sam"
    rgb_dir.mkdir(parents=True, exist_ok=True)
    depth_dir.mkdir(parents=True, exist_ok=True)
    crop_dir.mkdir(parents=True, exist_ok=True)

    for _, image in tqdm(images.items(), f"Rendering object crops using {device} images"):
        world_to_camera = image.world_to_camera

        rgb_rendered, _, vert_indices = render_engine.renderAll(world_to_camera, near, far)

        iphone_rgb_path = Path(scene.iphone_rgb_dir) / image.name
        rgb = np.asarray(imageio.imread(iphone_rgb_path))

        vert_instance = vert_to_obj[vert_indices]
        pix_instance = vert_instance[:, :, 0] # Some triangles actually belong to different objects. I don't think it will matter for crops.

        # Visualize instances
        # instance_rgb = instance_colors[pix_instance]
        # imageio.imwrite(rgb_dir / image.name, instance_rgb)

        objs = np.unique(pix_instance)

        for obj in objs:
            mask = pix_instance == obj
            crop = crop_rgb_mask(rgb, rgb_rendered, mask, inflate_px=100)
            crop_heaps[obj]["heap"].push(crop)


        # instance_rgb = instance_rgb.astype(np.uint8)
        # # Make depth in mm and clip to fit 16-bit image
        # depth = (depth.astype(np.float32) * 1000).clip(0, 65535).astype(np.uint16)
        # depth_name = image.name.split(".")[0] + ".png"
        # imageio.imwrite(depth_dir / depth_name, depth)

Init EGL
Detected 5 devices
Using device 0
Using EGL version 1.5
OpenGL version: 4.6.0 NVIDIA 550.107.02
EGL version: 1.5
Loaded mesh:MeshData:
	Vertices:  917079
	Colors:    917079
	Normals:   0
	TexCoords: 0


Copy mesh to GPU: 917079 vertices, 1832073 faces
Setup the frame and render buffer


Rendering object crops using iphone images:   0%|          | 0/586 [00:00<?, ?it/s]/tmp/ipykernel_151467/1520543007.py:61: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  rgb = np.asarray(imageio.imread(iphone_rgb_path))
Rendering object crops using iphone images: 100%|██████████| 586/586 [03:54<00:00,  2.50it/s]


In [7]:
# for id, entry in tqdm(crop_heaps.items(), f"Rendering image grids"):
#     heap = entry["heap"]
#     label = entry["label"]
#     if len(heap) and label.lower() not in [
#         "background",
#         "wall",
#         "floor",
#         "ceiling",
#         "split",
#         "remove",
#     ]:
#         crops = heap.get_sorted()
#         rgbs = [c.rgb for c in crops]
#         masks = [c.mask for c in crops]
#         scores = [c.score for c in crops]
#         plot_grid_images(
#             rgbs + masks, grid_width=len(rgbs), title=entry["label"]
#         )
#         plt.savefig(crop_dir / f"{str(id).zfill(5)}.jpg")
#         plt.close()

In [10]:
import torch
import os
import tempfile
import shutil
import cv2
from sam2.build_sam import build_sam2_video_predictor

class SAM2VideoMaskModel:
    def __init__(self, sam2_checkpoint, model_cfg, device="cuda"):
        """
        Initialize SAM2 model and create a temporary directory for storing video frames.

        Args:
            sam2_checkpoint (str): Path to the SAM2 checkpoint file.
            model_cfg (str): Path to the SAM2 model configuration file.
            device (str): Device to run SAM2 on (e.g., 'cuda' or 'cpu').
        """
        self.device = torch.device(device)
        if self.device.type == "cuda":
            torch.autocast("cuda", dtype=torch.bfloat16).__enter__()

        self.predictor = build_sam2_video_predictor(
            model_cfg, sam2_checkpoint, device=self.device
        )

        # Create a temporary directory for storing frames
        self.temp_dir = tempfile.mkdtemp()
        print(f"Temporary directory created at: {self.temp_dir}")

        # Placeholder for resized images, masks, padded_masks, rgb_padded, padding information, and scores
        self.rgb = []
        self.rgb_padded = []
        self.mask = []
        self.padded_masks = []
        self.scores = []
        self.padding_info = []
        self.refined_masks = []

    def pad_and_store(self, rgbs, masks, scores):
        """
        Pads the RGB images and masks to the size of the largest image and stores the RGBs in the temp directory.
        Also stores the provided scores, padded masks, and padding information for each image.

        Args:
            rgbs: List of RGB images in cv2 format (H, W, 3).
            masks: List of mask images in cv2 format (H, W) or (H, W, 1).
            scores: List of scores for each corresponding RGB and mask.
        """
        if len(rgbs) != len(masks) or len(rgbs) != len(scores):
            raise ValueError("The number of RGB images, masks, and scores must match.")

        # Determine the target size based on the largest dimensions of the images
        max_h = max([rgb.shape[0] for rgb in rgbs])  # Find the max height
        max_w = max([rgb.shape[1] for rgb in rgbs])  # Find the max width

        for idx, (rgb, mask, score) in enumerate(zip(rgbs, masks, scores)):
            # Calculate padding for RGB and mask to fit into the max dimensions
            h, w, _ = rgb.shape
            pad_h = max_h - h
            pad_w = max_w - w
            padding_info = (
                (0, pad_h),
                (0, pad_w),
            )  # Store padding applied for future unpadding

            # Apply padding to RGB and mask
            padded_rgb = np.pad(
                rgb,
                ((0, pad_h), (0, pad_w), (0, 0)),
                mode="constant",
                constant_values=0,
            )
            padded_mask = np.pad(
                mask, ((0, pad_h), (0, pad_w)), mode="constant", constant_values=0
            )

            # Save the padded RGB to the temp directory
            rgb_filename = os.path.join(self.temp_dir, f"{idx}.jpg")
            cv2.imwrite(rgb_filename, padded_rgb)

            # Store the original and padded images, masks, scores, and padding info
            self.rgb.append(rgb)
            self.rgb_padded.append(padded_rgb)
            self.mask.append(mask)
            self.padded_masks.append(padded_mask)
            self.scores.append(score)
            self.padding_info.append(padding_info)

        # print(f"Stored {len(self.rgb)} padded RGB images, masks, and scores.")

    def set_state_and_refine_masks(self):
        """
        Set the state for the SAM2 predictor, refine the masks based on the highest scoring image,
        and propagate the refined mask through all frames.
        """
        # Set the inference state using the temp_dir where padded frames are stored
        inference_state = self.predictor.init_state(video_path=self.temp_dir)

        # Find the RGB image with the highest score
        highest_score_idx = np.argmax(self.scores)
        highest_score_rgb = self.rgb_padded[highest_score_idx]
        highest_score_mask = self.padded_masks[highest_score_idx]

        # Sample two points for initialization (centroid and another random point in the mask)
        mask_indices = np.argwhere(
            highest_score_mask > 0
        )  # Get the indices of the mask
        # centroid = np.mean(mask_indices, axis=0).astype(int)
        random_point_1 = mask_indices[np.random.choice(len(mask_indices))]
        random_point_2 = mask_indices[np.random.choice(len(mask_indices))]

        points = np.array([random_point_1, random_point_2], dtype=np.float32)
        labels = np.array([1, 1], dtype=np.int32)  # Positive clicks for both points

        # Initialize the predictor with points and mask
        _, out_obj_ids, out_mask_logits = self.predictor.add_new_points_or_box(
            inference_state=inference_state,
            frame_idx=highest_score_idx,  # Use the index of the highest score frame
            obj_id=1,  # Give a unique ID to this object
            points=points,
            labels=labels,
            box=None,  # No box used in this example
        )

        # Propagate the refined mask through all frames and store them
        for out_frame_idx, out_obj_ids, out_mask_logits in self.predictor.propagate_in_video(inference_state):
            # We assume that there is only one object per frame, so we can extract the first mask
            refined_mask = (out_mask_logits[0] > 0.0).cpu().numpy().squeeze()
            self.refined_masks.append(refined_mask)

        # print("Refined masks propagated and stored for all frames.")

        self.predictor.reset_state(inference_state)
        # print("Inference state reset.")

    def unpad_masks_to_original_size(self):
        """
        Converts the refined masks back to their original sizes by removing the padding.
        Assumes `self.refined_masks` is a list of masks with shape (h, w).
        """
        original_size_masks = []
        
        for frame_idx, mask in enumerate(self.refined_masks):
            # Retrieve the padding information for this frame
            pad_h, pad_w = self.padding_info[frame_idx]  # Get padding for the frame

            # Remove padding from the mask
            if pad_h[1] != 0 or pad_w[1] != 0:
                original_mask = mask[:-pad_h[1], :-pad_w[1]]
            else:
                original_mask = mask  # No padding, keep the mask as it is
            
            # Append the unpadded mask to the original_size_masks list
            original_size_masks.append(original_mask)

        self.refined_masks = original_size_masks  # Update the refined masks to their original sizes
        # print("Masks resized back to original dimensions.")

    def cleanup(self):
        """
        Cleans up the temporary directory and clears all stored data except the SAM2 model.
        """
        if os.path.exists(self.temp_dir):
            for filename in os.listdir(self.temp_dir):
                file_path = os.path.join(self.temp_dir, filename)
                try:
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.unlink(file_path)  # Remove the file or symbolic link
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)  # Remove the subdirectory and its contents
                except Exception as e:
                    print(f"Failed to delete {file_path}. Reason: {e}")

            print(f"Cleared contents of temporary directory {self.temp_dir}.")

        # Clear all stored data (RGBs, masks, etc.) except the SAM2 model
        self.rgb = []
        self.rgb_padded = []
        self.mask = []
        self.padded_masks = []
        self.scores = []
        self.padding_info = []
        self.refined_masks = []
        
        print("Cleared all stored images, masks, scores, and padding information.")


In [11]:
sam2_checkpoint = "/home/kumaraditya/checkpoints/sam2_hiera_large.pt"
sam2_model_cfg = "sam2_hiera_l.yaml"
sam2_model = SAM2VideoMaskModel(sam2_checkpoint, sam2_model_cfg)


Temporary directory created at: /tmp/tmpehswpppo


In [12]:
for id, entry in tqdm(crop_heaps.items(), f"Rendering image grids"):
    heap = entry["heap"]
    label = entry["label"]
    if len(heap) and label.lower() not in [
        "background",
        "wall",
        "floor",
        "ceiling",
        "split",
        "remove",
    ]:
        crops = heap.get_sorted()
        rgbs = [c.rgb for c in crops]
        masks = [c.mask for c in crops]
        scores = [c.score for c in crops]

        sam2_model.pad_and_store(rgbs, masks, scores)
        sam2_model.set_state_and_refine_masks()
        sam2_model.unpad_masks_to_original_size()
        masks = sam2_model.refined_masks
        sam2_model.cleanup()
        
        plot_grid_images(
            rgbs + masks, grid_width=len(rgbs), title=entry["label"]
        )
        plt.savefig(crop_dir / f"{str(id).zfill(5)}.jpg")
        plt.close()

frame loading (JPEG): 100%|██████████| 4/4 [00:00<00:00, 33.66it/s]
/home/kumaraditya/segment-anything-2/sam2/sam2_video_predictor.py:873: UserWarning: cannot import name '_C' from 'sam2' (/home/kumaraditya/segment-anything-2/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 4/4 [00:01<00:00,  3.85it/s]
/home/kumaraditya/scannetpp/common/render_crops_utils.py:185: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  ax.imshow(img)
/home/kumaraditya/scannetpp/common/render_crops_utils.py:185: UserWarning: Attempting to set identical low and high ylims makes transform

Cleared contents of temporary directory /tmp/tmpehswpppo.
Cleared all stored images, masks, scores, and padding information.


frame loading (JPEG): 100%|██████████| 4/4 [00:00<00:00, 42.06it/s]it]
/home/kumaraditya/segment-anything-2/sam2/sam2_video_predictor.py:873: UserWarning: cannot import name '_C' from 'sam2' (/home/kumaraditya/segment-anything-2/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 4/4 [00:01<00:00,  3.98it/s]
/home/kumaraditya/scannetpp/common/render_crops_utils.py:185: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.imshow(img)
/home/kumaraditya/scannetpp/common/render_crops_utils.py:185: UserWarning: Attempting to set identical low and high xlims makes transf

Cleared contents of temporary directory /tmp/tmpehswpppo.
Cleared all stored images, masks, scores, and padding information.


frame loading (JPEG): 100%|██████████| 4/4 [00:00<00:00, 38.90it/s]it]
/home/kumaraditya/segment-anything-2/sam2/sam2_video_predictor.py:873: UserWarning: cannot import name '_C' from 'sam2' (/home/kumaraditya/segment-anything-2/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 4/4 [00:01<00:00,  3.97it/s]
/home/kumaraditya/scannetpp/common/render_crops_utils.py:185: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  ax.imshow(img)
/home/kumaraditya/scannetpp/common/render_crops_utils.py:185: UserWarning: Attempting to set identical low and high ylims makes transf

...

frame loading (JPEG): 100%|██████████| 4/4 [00:00<00:00, 33.42it/s]s/it]
/home/kumaraditya/segment-anything-2/sam2/sam2_video_predictor.py:873: UserWarning: cannot import name '_C' from 'sam2' (/home/kumaraditya/segment-anything-2/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 4/4 [00:01<00:00,  3.42it/s]
/home/kumaraditya/scannetpp/common/render_crops_utils.py:185: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.imshow(img)
/home/kumaraditya/scannetpp/common/render_crops_utils.py:185: UserWarning: Attempting to set identical low and high xlims makes tran

Cleared contents of temporary directory /tmp/tmpehswpppo.
Cleared all stored images, masks, scores, and padding information.


frame loading (JPEG): 100%|██████████| 4/4 [00:00<00:00, 28.91it/s]s/it]
/home/kumaraditya/segment-anything-2/sam2/sam2_video_predictor.py:873: UserWarning: cannot import name '_C' from 'sam2' (/home/kumaraditya/segment-anything-2/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 4/4 [00:01<00:00,  3.57it/s]


Cleared contents of temporary directory /tmp/tmpehswpppo.
Cleared all stored images, masks, scores, and padding information.


Rendering image grids: 100%|██████████| 135/135 [03:46<00:00,  1.68s/it]
